In [ ]:
import os.path
import pandas as pd
import numpy as np
from matplotlib import pyplot

from settings.settings import HISTORIC_DATA_FOLDER, ELECTRICITY_PER_QUARTER_FILE, GAS_PER_DAY_FILE

%config IPCompleter.greedy=True
# %matplotlib widget
# pd.set_option('display.max_rows', None) # show all rows in dataframe

In [ ]:
def get_data_as_df(csv_file_name):
    dt_concat_format = '%d-%m-%Y %H:%M:%S'
    util_df = pd.read_csv(csv_file_name, sep=';', decimal=',')
    util_df['End Time'] = util_df['Tot Datum'] + " " + util_df['Tot Tijdstip']
    util_df['End Time'] = pd.to_datetime(util_df['End Time'], format=dt_concat_format)
    util_df['Start Time'] = util_df['Van Datum'] + " " + util_df['Van Tijdstip']
    util_df['Start Time'] = pd.to_datetime(util_df['Start Time'], format=dt_concat_format)
    util_df['DT delta'] = util_df['End Time'] - util_df['Start Time']
    util_df.drop(columns=['EAN', 'Meter', 'Metertype', 'Validatiestatus', 'Tot Datum', 'Tot Tijdstip', 'Van Datum', 'Van Tijdstip'], inplace=True)
    util_df.set_index('End Time', inplace=True)

    return util_df

In [ ]:
def add_week_number_column(df_in, year):
    df_in = df_in.loc[year]
    df_in['Week Number'] = df_in.index.isocalendar().week
    df_in.set_index('Week Number', inplace=True)
    
    return df_in

In [ ]:
def add_doy_column(df_in, year):
    df_in = df_in.loc[year]
    df_in['DOY'] = df_in.index.dayofyear
    df_in.set_index('DOY', inplace=True)
    
    return df_in

In [ ]:
df_gas = get_data_as_df(os.path.join(HISTORIC_DATA_FOLDER, GAS_PER_DAY_FILE))
m3_filter = df_gas['Eenheid'].isin(['m³'])
df_gas = df_gas[m3_filter]

In [ ]:
expected_gas_interval = np.timedelta64(1, 'D')
unexpected_interval = df_gas.loc[df_gas['DT delta'] != expected_gas_interval]
print('!! be aware !!')
print('unexpected interval, possibly caused by summer/winter time')
unexpected_interval

In [ ]:
gpd = df_gas.resample('d').sum(numeric_only=True)
gpw = df_gas.resample('w').sum(numeric_only=True)

In [ ]:
fig, axes = pyplot.subplots(figsize=(40, 8))
ax = df_gas['Volume'].plot(label='per day', grid=True)
bx = gpw['Volume'].plot(label='per week', grid=True, style='o-')
bx.set_ylabel('gas volume (m³)')
axes.legend()
pyplot.show()

In [ ]:
gpw_2023 = add_week_number_column(gpw, '2023')
gpw_2022 = add_week_number_column(gpw, '2022')
gpw_2021 = add_week_number_column(gpw, '2021')

# last week of 2021 is according to pandas in the year 2022. Move it to 2021, to prevent weird plotting.
gpw_2021.loc[52] = gpw_2022.loc[52]
gpw_2022.drop(52, inplace=True)

# first week of 2023 is according to pandas in the year 2022. Move it back, to prevent weird plotting.
gpw_2022.loc[52] = gpw_2023.loc[52]
gpw_2023.drop(52, inplace=True)

In [ ]:
fig, axes = pyplot.subplots(sharex=True, figsize=(40, 8))
ax = gpw_2021['Volume'].plot(label='2021', grid=True, style='d--')
bx = gpw_2022['Volume'].plot(label='2022', grid=True, style='o-')
cx = gpw_2023['Volume'].plot(label='2023', grid=True, style='cx-')
cx.set_ylabel('gas volume (m³)')

axes.set_title('Gas volume per week - year by year comparison')
axes.legend()
pyplot.show()

In [ ]:
gpd_2021 = add_doy_column(gpd, '2021')
gpd_2022 = add_doy_column(gpd, '2022')
gpd_2023 = add_doy_column(gpd, '2023')

In [ ]:
# fig, axes = pyplot.subplots(sharex=True, figsize=(40, 8))
# ax = gpd_2021['Volume'].plot(label='2021', grid=True, style='d--')
# bx = gpd_2022['Volume'].plot(label='2022', grid=True, style='o-')
# cx = gpd_2023['Volume'].plot(label='2023', grid=True, style='cx-')
# cx.set_ylabel('gas volume (m³)')

# # mark September 1st
# bx.annotate("", xy=(243, 2), xytext=(243, 0), arrowprops=dict(arrowstyle="-"))

# axes.set_title('Gas per day - DOY comparison')
# axes.legend()
# pyplot.show()

In [ ]:
# day 90 = April 1; day 242 = September 1.
cut_off_day = 242
gpd_2021_last_months = gpd_2021.loc[gpd_2021.index > cut_off_day]
gpd_2022_last_months = gpd_2022.loc[gpd_2022.index > cut_off_day]
gpd_2023_last_months = gpd_2023.loc[gpd_2023.index > cut_off_day]
cumulative_2021 = gpd_2021_last_months['Volume'].cumsum()
cumulative_2022 = gpd_2022_last_months['Volume'].cumsum()
cumulative_2023 = gpd_2023_last_months['Volume'].cumsum()

In [ ]:
last_day = 34#cumulative_2022.size - 1 
print(f'day:\t{last_day}')
print(f'2021:\t{cumulative_2021.iloc[last_day]:.1f} m³')
print(f'2022:\t{cumulative_2022.iloc[last_day]:.1f} m³')
print(f'2023:\t{cumulative_2023.iloc[last_day]:.1f} m³')

In [ ]:
# fig, axes = pyplot.subplots(sharex=True, figsize=(40, 8))
# ax = gpd_2021_last_months['Volume'].plot(label='2021', grid=True, style='d--')
# bx = gpd_2022_last_months['Volume'].plot(label='2022', grid=True, style='o-')

# bx.set_ylabel('gas volume (m³)')
# axes.set_title('Last X months - gas per day')
# axes.legend()
# pyplot.show()

In [ ]:
fig, axes = pyplot.subplots(sharex=True, figsize=(40, 8))
ax = cumulative_2021.plot(label='2021', grid=True, style='d--')
bx = cumulative_2022.plot(label='2022', grid=True, style='o-')
cx = cumulative_2023.plot(label='2023', grid=True, style='+-')

cx.set_ylabel('gas volume (m³)')
axes.set_title('Last X months - gas cumulative by day')
axes.legend()
pyplot.show()

In [ ]:
df_elec = get_data_as_df(os.path.join(HISTORIC_DATA_FOLDER, ELECTRICITY_PER_QUARTER_FILE))

afname_filter = df_elec['Register'].isin(['Afname Dag', 'Afname Nacht'])
elec_afname = df_elec.loc[afname_filter]

injectie_filter = df_elec['Register'].isin(['Injectie Dag', 'Injectie Nacht'])
elec_injectie = df_elec.loc[injectie_filter]

expected_elec_interval = np.timedelta64(15, 'm')
unexpected_electr_interval = df_elec.loc[df_elec['DT delta'] != expected_elec_interval]
print('!! be aware !!')
print('unexpected interval, possibly caused by summer/winter time')
unexpected_electr_interval

In [ ]:
# fig, axes = pyplot.subplots(figsize=(40, 8))
# ax = elec_injectie['Volume'].plot(grid=True, label='injectie')
# bx = elec_afname['Volume'].plot(grid=True, label='afname')
# axes.legend()
# pyplot.show()

In [ ]:
eapd = elec_afname.resample('d').sum(numeric_only=True)
eapw = elec_afname.resample('w').sum(numeric_only=True)

In [ ]:
fig, axes = pyplot.subplots(figsize=(40, 8))
ax = eapd['Volume'].plot(label='elec per day', grid=True)
ax.set_ylabel('elec afname kWh')
bx = eapw['Volume'].plot(label='per week', grid=True, style='o-')
axes.set_title('Electriciteit afname (per dag en per week)')
axes.legend()
pyplot.show()